In [1]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.ZHFx9Cf4Tc/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.TiBIsMfvux/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.JOzjNkzBw3/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.ubuntu.com/ubuntu

In [74]:
# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [76]:
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
 
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [ ]:
search_word = '갑오'
url = "http://www.davincimap.co.kr/davBase/Source/davSourList.jsp?job=S&worder=TITLE&wpara={}&wtype=2".format(search_word)
print(url)
driver.get(url)

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH,"/html/body/div[2]/div/div[2]/table[5]/tbody/tr[2]/td[2]/div/span"))
)

html = driver.page_source
print(html)

In [78]:
text_file = open("/content/output.html", "w",encoding='utf-8')
text_file.write(html)
text_file.close()



In [79]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import csv

In [80]:
soup = BeautifulSoup(html, 'html.parser')


In [81]:
table = soup.select('body > div.body_container > div > div.body_3Col_Main > table:nth-child(10)')
print(type(table), table)

<class 'bs4.element.ResultSet'> [<table border="0" cellpadding="0" colspan="0" height="50" width="700">
<tbody><tr align="center" width="98%">
</tr><tr align="center" height="20" rowspan="2" width="98%">
<td align="center" width="30">
<div style="font-size: 0.9em;font-weight: bold;">1</div> </td>
<td align="center" width="140">
<div style="color:OrangeRed;font-size: 0.9em;font-weight: bold;">SOUR004795</div> </td>
<td align="left" width="480">
<div style="font-size: 0.9em;"><a href="/davBase/Source/davSource.jsp?Job=Body&amp;SourID=SOUR004795"><span style="color:DarkBlue;font-weight: bold;">갑오 광복절</span></a>
</div> </td>
<td align="center" colspan="2" width="140">
<div style="font-size: 0.8em;"><font color="DarkRed">최남선</font> (1954)</div> </td>
</tr>
<tr align="center" height="30" valign="top" width="98%">
<td></td>
<td align="center" width="140">
<div style="font-size: 0.9em;padding:3px;">문학 &gt; 시</div>
<div style="font-size: 1.0em;padding:3px;"><span>(</span><span style="color:Dark

In [82]:
table = soup.select('body > div.body_container > div > div.body_3Col_Main > table:nth-child(10)')[0]
print(type(table), table)

<class 'bs4.element.Tag'> <table border="0" cellpadding="0" colspan="0" height="50" width="700">
<tbody><tr align="center" width="98%">
</tr><tr align="center" height="20" rowspan="2" width="98%">
<td align="center" width="30">
<div style="font-size: 0.9em;font-weight: bold;">1</div> </td>
<td align="center" width="140">
<div style="color:OrangeRed;font-size: 0.9em;font-weight: bold;">SOUR004795</div> </td>
<td align="left" width="480">
<div style="font-size: 0.9em;"><a href="/davBase/Source/davSource.jsp?Job=Body&amp;SourID=SOUR004795"><span style="color:DarkBlue;font-weight: bold;">갑오 광복절</span></a>
</div> </td>
<td align="center" colspan="2" width="140">
<div style="font-size: 0.8em;"><font color="DarkRed">최남선</font> (1954)</div> </td>
</tr>
<tr align="center" height="30" valign="top" width="98%">
<td></td>
<td align="center" width="140">
<div style="font-size: 0.9em;padding:3px;">문학 &gt; 시</div>
<div style="font-size: 1.0em;padding:3px;"><span>(</span><span style="color:DarkBlue;">

In [83]:
table = soup.select('body > div.body_container > div > div.body_3Col_Main > table:nth-child(10)')[0]
tr_list = table.find('tbody').find_all('tr')
print(type(tr_list), tr_list)

<class 'bs4.element.ResultSet'> [<tr align="center" width="98%">
</tr>, <tr align="center" height="20" rowspan="2" width="98%">
<td align="center" width="30">
<div style="font-size: 0.9em;font-weight: bold;">1</div> </td>
<td align="center" width="140">
<div style="color:OrangeRed;font-size: 0.9em;font-weight: bold;">SOUR004795</div> </td>
<td align="left" width="480">
<div style="font-size: 0.9em;"><a href="/davBase/Source/davSource.jsp?Job=Body&amp;SourID=SOUR004795"><span style="color:DarkBlue;font-weight: bold;">갑오 광복절</span></a>
</div> </td>
<td align="center" colspan="2" width="140">
<div style="font-size: 0.8em;"><font color="DarkRed">최남선</font> (1954)</div> </td>
</tr>, <tr align="center" height="30" valign="top" width="98%">
<td></td>
<td align="center" width="140">
<div style="font-size: 0.9em;padding:3px;">문학 &gt; 시</div>
<div style="font-size: 1.0em;padding:3px;"><span>(</span><span style="color:DarkBlue;">편</span><span> / </span><span style="color:DarkBlue;">게재</span><span

In [ ]:
table = soup.select('body > div.body_container > div > div.body_3Col_Main > table:nth-child(10)')[0]
tr_list = table.find('tbody').find_all('tr')
for tr in tr_list:
  print("------------")
  print(tr)
  print("------------")

In [85]:
table = soup.select('body > div.body_container > div > div.body_3Col_Main > table:nth-child(10)')[0]
tr_list = table.find('tbody').find_all('tr')
for tr in tr_list:
  a_tag = tr.find('a')
  if a_tag is None:
    continue
  print(a_tag)
  print(a_tag.text)
  print(a_tag.get('href'))
  source_link = 'http://www.davincimap.co.kr'+a_tag.get('href')
  print(source_link)
  print("-----------------")


<a href="/davBase/Source/davSource.jsp?Job=Body&amp;SourID=SOUR004795"><span style="color:DarkBlue;font-weight: bold;">갑오 광복절</span></a>
갑오 광복절
/davBase/Source/davSource.jsp?Job=Body&SourID=SOUR004795
http://www.davincimap.co.kr/davBase/Source/davSource.jsp?Job=Body&SourID=SOUR004795
-----------------
<a href="/davBase/Source/davSource.jsp?Job=Body&amp;SourID=SOUR004796"><span style="color:DarkBlue;font-weight: bold;">갑오 삼일절</span></a>
갑오 삼일절
/davBase/Source/davSource.jsp?Job=Body&SourID=SOUR004796
http://www.davincimap.co.kr/davBase/Source/davSource.jsp?Job=Body&SourID=SOUR004796
-----------------
<a href="/davBase/Source/davSource.jsp?Job=Body&amp;SourID=SOUR004685"><span style="color:DarkBlue;font-weight: bold;">갑오비화</span> <span>(甲午秘話)</span></a>
갑오비화 (甲午秘話)
/davBase/Source/davSource.jsp?Job=Body&SourID=SOUR004685
http://www.davincimap.co.kr/davBase/Source/davSource.jsp?Job=Body&SourID=SOUR004685
-----------------


In [86]:
today = datetime.datetime.now()
time_str = today.strftime("%Y-%m-%d %H:%M:%S")
print(time_str)

2023-06-12 03:56:46


In [87]:
fd = open('/content/output.tsv', 'w', encoding='utf-8', newline='')
wr = csv.writer(fd,delimiter='\t')
wr.writerow(['keyword','title','source','time'])


table = soup.select('body > div.body_container > div > div.body_3Col_Main > table:nth-child(10)')[0]
tr_list = table.find('tbody').find_all('tr')
for tr in tr_list:
  a_tag = tr.find('a')
  if a_tag is None:
    continue
  source_link = 'http://www.davincimap.co.kr'+a_tag.get('href')

  today = datetime.datetime.now()
  time_str = today.strftime("%Y-%m-%d %H:%M:%S")
  row = []
  row.append(search_word)
  row.append(a_tag.text)  
  row.append(source_link)
  row.append(time_str)
  wr.writerow(row)

fd.close()


In [96]:
fd = open('/content/output.tsv', 'r', encoding='utf-8')
data = csv.reader(fd,delimiter='\t')
next(data)
text_index = 0
for line in data:
  text_index += 1
  print(line[2])
  driver.get(line[2])

  WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.XPATH,"/html/body/table[1]/tbody/tr/td[1]/table[4]"))
  )

  text_html = driver.page_source
  
  
  text_file = open("/content/"+str(text_index)+".html", "w",encoding='utf-8')
  text_file.write(text_html)
  text_file.close()

  if True:
    break

fd.close()

http://www.davincimap.co.kr/davBase/Source/davSource.jsp?Job=Body&SourID=SOUR004795


In [ ]:
fd = open("1.html", "rt", encoding='utf-8')
page = fd.read()
soup = BeautifulSoup(page, 'html.parser')
table = soup.select('#myDIVMain > table:nth-child(8) > tbody > tr:nth-child(1) > td > table > tbody > tr > td')[0]

for d in table.find_all('div'):
  if d.text.strip() == '':
    continue
  print('----------')
  print(d.text)

fd.close()